In [20]:
import yfinance as yf
import pandas as pd

def calculate_roic(financials, balance_sheet):
    try:
        # Print columns and data in the balance sheet
        print("Balance Sheet Columns and Data:")
        print(balance_sheet)
        
        # Flexible key matching for 'Total Assets' and 'Total Current Liabilities'
        total_assets_col = [col for col in balance_sheet.columns if 'Total Assets' in col]
        current_liabilities_col = [col for col in balance_sheet.columns if 'Total Current Liabilities' in col]
        
        if total_assets_col and current_liabilities_col:
            total_assets = balance_sheet[total_assets_col[0]].iloc[0]
            total_current_liabilities = balance_sheet[current_liabilities_col[0]].iloc[0]
            net_income = financials['Net Income'].iloc[0]
            
            if total_assets == 0 or (total_assets - total_current_liabilities) == 0:
                print("Cannot calculate ROIC: Total Assets - Total Current Liabilities equals 0.")
                return None
            
            invested_capital = total_assets - total_current_liabilities
            roic = net_income / invested_capital
            return roic
        else:
            raise KeyError("Required data for ROIC calculation is missing.")
    except KeyError as e:
        print(f"Missing key data for ROIC calculation: {e}")
        return None

def calculate_fcf_growth(cashflow):
    try:
        # Print columns and data in the cashflow
        print("Cash Flow Columns and Data:")
        print(cashflow)
        
        if 'Free Cash Flow' in cashflow:
            fcf = cashflow['Free Cash Flow'].infer_objects()  # Ensure correct object types
            fcf_growth = fcf.pct_change().mean() * 100
            return fcf_growth
        else:
            raise KeyError("Free Cash Flow data is missing.")
    except KeyError as e:
        print(f"Missing key data for FCF growth calculation: {e}")
        return None

def calculate_profit_margins(financials):
    try:
        # Print columns and data in the financials
        print("Financials Columns and Data:")
        print(financials)
        
        if 'Net Income' in financials and 'Revenue' in financials and 'Gross Profit' in financials:
            net_income = financials['Net Income'].iloc[0]
            revenue = financials['Revenue'].iloc[0]
            gross_profit = financials['Gross Profit'].iloc[0]

            net_margin = (net_income / revenue) * 100
            gross_margin = (gross_profit / revenue) * 100
            return net_margin, gross_margin
        else:
            raise KeyError("Required data for profit margin calculation is missing.")
    except KeyError as e:
        print(f"Missing key data for profit margin calculation: {e}")
        return None

def fetch_company_data(ticker):
    stock = yf.Ticker(ticker)
    financials = stock.financials.T  # Financial statements
    balance_sheet = stock.balance_sheet.T  # Balance sheet
    cashflow = stock.cashflow.T  # Cash flow statement
    
    # Print all fetched data to understand the structure
    print("Fetched Financials Data:")
    print(financials)
    print("\nFetched Balance Sheet Data:")
    print(balance_sheet)
    print("\nFetched Cash Flow Data:")
    print(cashflow)

    return financials, balance_sheet, cashflow

def analyze_company_moat(ticker):
    # Fetch data from yfinance
    financials, balance_sheet, cashflow = fetch_company_data(ticker)

    # Calculate moat-related metrics
    roic = calculate_roic(financials, balance_sheet)
    fcf_growth = calculate_fcf_growth(cashflow)
    profit_margins = calculate_profit_margins(financials)

    if profit_margins is not None:
        net_margin, gross_margin = profit_margins
    else:
        net_margin, gross_margin = None, None

    if roic is not None:
        print(f"ROIC: {roic:.2f}")
    else:
        print("ROIC could not be calculated.")

    if fcf_growth is not None:
        print(f"Free Cash Flow Growth: {fcf_growth:.2f}%")
    else:
        print("FCF growth could not be calculated.")

    if net_margin is not None and gross_margin is not None:
        print(f"Net Margin: {net_margin:.2f}%")
        print(f"Gross Margin: {gross_margin:.2f}%")
    else:
        print("Profit margins could not be calculated.")

    # Example of how you might conclude moat analysis
    if roic is not None and roic > 10 and fcf_growth is not None and fcf_growth > 5:
        print("The company has a significant moat.")
    else:
        print("The company has no significant moat.")

# Example: Analyze a company (replace with any ticker)
analyze_company_moat('AAPL')  # Apple example

Fetched Financials Data:
           Tax Effect Of Unusual Items Tax Rate For Calcs Normalized EBITDA  \
2023-09-30                         0.0              0.147    129188000000.0   
2022-09-30                         0.0              0.162    133138000000.0   
2021-09-30                         0.0              0.133    123136000000.0   
2020-09-30                         0.0              0.144     81020000000.0   

           Net Income From Continuing Operation Net Minority Interest  \
2023-09-30                                      96995000000.0           
2022-09-30                                      99803000000.0           
2021-09-30                                      94680000000.0           
2020-09-30                                      57411000000.0           

           Reconciled Depreciation Reconciled Cost Of Revenue          EBITDA  \
2023-09-30           11519000000.0             214137000000.0  129188000000.0   
2022-09-30           11104000000.0             2235